# <big><font color='#D35400'>Crear múltiples crossplots Densidad-Neutrón utilizando plotly</font><big>

In [ ]:
import lasio
import pandas as pd
import os
from IPython.display import clear_output

import plotly.express as px

<a id='carga'></a>
# <font color='#8E44AD'>CARGA DE DATOS</font>

In [ ]:
if 'lista_pozos' in locals():
    del lista_pozos
else:
    lista_pozos = []
    
lista_pozos = []   
path=input("Escriba el PATH completo de la carpeta que contiene los archivos .las y presione ENTER. (Ejemplo: D:\Pablo\Python\working_files)")
clear_output(wait=True)
# D:\Pablo\Python\working_files\pozos

for file in os.listdir(path):
    if file.endswith(".las"):
        print(os.path.join(file))

In [ ]:
pozo_nomb_comienzo=input("Escriba las primeras letras/números de los archivos .las con el que va a trabajar y presione ENTER,si va a utilizar todos los .las de la carpeta, solo presione ENTER")

clear_output(wait=True)

for pozo in os.listdir(path):
    if pozo.endswith('.las') and pozo.startswith((pozo_nomb_comienzo.lower(),pozo_nomb_comienzo.upper(),pozo_nomb_comienzo.capitalize())):
        lista_pozos.append(path+"\\"+pozo)
        
lista_pozos

## <font color='#2F60EB'>Crear 1 df con los datos de todos los pozos</font>

In [ ]:
df_lista_pozos = []

for pozo in lista_pozos:
    las = lasio.read(pozo)
    df_las = las.df()
    df_las['DEPTH'] = df_las.index
    df_las['WELL'] = las.well.WELL.value
    df_las['WELL_TOP'] = las.well.STRT.value
    df_las['WELL_BOT'] = las.well.STOP.value
    df_lista_pozos.append(df_las)
    
df_pozos = pd.concat(df_lista_pozos, sort=True)
pd.set_option('display.max_columns', None)

print ('\033[1m'+'POZOS:')
for uniq in sorted(df_pozos["WELL"].unique()):
    print (uniq)

df_pozos

In [ ]:
df_pozos.describe()

In [ ]:
df_pozos.groupby('WELL').mean()

In [ ]:
df_pozos.groupby('WELL').mean()[['WELL_TOP','WELL_BOT']]

<a id='px'></a>
# <font color='#8E44AD'>SCATTER PLOT EN PLOTLY EXPRESS</font>

In [ ]:
################################### DATOS ####################################
##### Columnas #####
crv_x = 'NPHI'
crv_y = 'RHOB'
crv_color= 'GR'
crv_facet = 'WELL'
crv_size = 'CALI'

### Escalas curvas ###
crv_x_izq = 0
crv_x_der = 0.8

crv_y_aba = 3.2
crv_y_arr = 1.2

color_izq = 0
color_der = 120
color_cmap = px.colors.sequential.Hot_r

################################### CODIGO ####################################

##### Creamos un df solo con las curvas del grafico + DETPH #####
plotdf = pd.DataFrame()
crv_copy= df_pozos[[crv_x,crv_y,crv_color,crv_facet,crv_size,'DEPTH']]
plotdf = crv_copy.copy(deep=True)

df = plotdf[df_pozos[crv_size].notnull()]     

##### PLOT #####
fig = px.scatter(df, x=df[crv_x], y=df[crv_y],
                 color=df[crv_color],range_color=[color_izq,color_der], color_continuous_scale=color_cmap,
                 size=df[crv_size],
                 facet_col=df[crv_facet],facet_col_wrap=2)

fig.update_layout(autosize=False, width=1000, height=1200)
fig.update_coloraxes(colorbar_len=0.5)  
fig.update_xaxes(range=[crv_x_izq, crv_x_der])
fig.update_yaxes(range=[crv_y_aba, crv_y_arr])
fig.show()